# Defining our API

```
openapi: 3.2.0
info:
  title: Soil API
  version: 1.0.0
  description: |
    This API provides access to soil data provided by the NRCS.
    Soil Survey Staff, Natural Resources Conservation Service, United States Department of Agriculture. Soil Survey Geographic (SSURGO) Database for the USA. Available online. Accessed [10/28/2025].

servers:
  - url: https://api.dirtio.com
```

This is the most important step when building an API is system design. It's a lot easier to build something when it's been defined. Starting with user actions and creating mock cURLs:

## Users want to click on a portion of the map, and received soil characteristics of that parcel.
A click on the map is going to produce a coordinate. That coordinate can be used to get both the mapunit for the parcel (thereby getting any soil data related to the parcel) as well as the shape of the parcel (we can draw the parcel on the map!)

### Naive Design
This means our call is sending coordinates and receiving: a polygon, a mapunit key, and all data associated with that mapunit key.

So, that means:
```
GET /soil?lat=38.5&lon=-121.7

{mapunit: "12345543", shape: "POLYGON ((-122.407560312536 37.4779244261786, -122.407733973112 37.4780814925273, -122.407862057399 37.4781364448679, -122.408171588894 37.4781583629602, -122.408292518557 37.4786946355015))", ...}
```
This is a little big for one call. So, let's break it apart and assess some tradeoffs, with this on one end and making every single atomic call on the other end.

### Breaking Up the Call Atomically
We can send coordinates and do the following:
  1. Get the mapunit key
  2. Get the mapunit's polygon key
  3. Get all the polygons themselves
  4. Get the soil data

Once we have the mapunit key and polygon key, we can do the following:
  1. Filter out the polygons until we get the one assoicated with the coordinate.
  2. Get the soil data of the polyon key

On the backend, we're calling the NRCS webservice, which has rate limiting associated with it. Not to mention, the calls we'll be making are SQL statements that we're sending to the NRCS Database.

Every `get` listed is a database query. At this point in time, especially v1, we should make one call and iterate from there depending on user feedback.
